In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pandas as pnd

encoder = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
labelEncoder = LabelEncoder()



dadosDeTreino = pnd.read_csv('train.csv')
labelEncoder.fit_transform(dadosDeTreino['explicit'])
encodeFeats = ['album_name', 'track_genre']
encoded_data = encoder.fit_transform(dadosDeTreino[encodeFeats])
dadosDeTreino = pnd.concat([dadosDeTreino, encoded_data], axis=1).drop(columns=encodeFeats)
x=dadosDeTreino.drop(['popularity_target', 'track_unique_id','track_id','artists','track_name'], axis=1)
y=dadosDeTreino['popularity_target']

A preço de curiosidade, meu pc simplesmente não tem ram pra rodar isso ai (64 gigas não é o suficiente :O)

In [9]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.1, random_state=42)

In [ ]:
classifier = ExtraTreesClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
classifier.fit(x_treino, y_treino)
y_pred = classifier.predict(x_teste)
print('****************************************************************************************')
print(classification_report(y_teste, y_pred))
print('****************************************************************************************')
print(confusion_matrix(y_teste, y_pred))

In [ ]:
modelos_base = [('modelo9', ExtraTreesClassifier(criterion='gini', max_depth=None, max_features=None, n_estimators=1000, n_jobs=-1)),('modelo8', RandomForestClassifier(bootstrap=True, criterion='gini',  max_depth=None, max_features=None, n_estimators=1000, n_jobs=-1, random_state=42)),('modelo2',RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=20, max_features=None, n_estimators=500, n_jobs=-1, random_state=42))]
meta_modelo = SVC()
modelo_stack = StackingClassifier(estimators=modelos_base, final_estimator=meta_modelo, cv=5)
modelo_stack.fit(x_treino, y_treino)
print("*******************************************************")
print(accuracy_score(y_teste, modelo_stack.predict(x_teste)))
print(classification_report(y_teste, modelo_stack.predict(x_teste)))

print("*******************************************************")

In [4]:
df = pnd.read_csv('test.csv')
df.dropna(inplace=True)
encoder.fit(df['explicit'])
df['explicit'] = encoder.transform(df['explicit'])
encoder.fit(df['track_genre'])
df['track_genre'] = encoder.transform(df['track_genre'])
x=df[[ 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']]
df['popularity_target'] = modelo_stack.predict(x)
df[['track_unique_id', 'popularity_target']].to_csv('submission7.csv', index=False)